# Feliz Mancala
### Caleb Anderson, Aiden Devine

# Libraries, Frameworks, and Update #
We used the aima-python library (https://github.com/aimacode/aima-python.git) for our minimax and alpha-beta pruning algorithm framework. \
For ease of implementation, we copied the functions from the games4e.py file and modified them to fit our Mancala class and it's requirements. 

So far, we have gotten the minmax_decision function working. \
We still need to implement alpha_beta_cutoff_search function which should be straightforward and similar to the min_max function.

## Mancala rules to be followed 
**(there are few modifications from the original game, please read this before writing the code)**

- On every turn, select a pit and distribute its stones in a counter-clockwise direction.
    - If the last stone lands in the player's mancala, in an opponent's pit, or in one of the player's non-empty pits, no further action is taken, and the current player's turn ends.
    - If the last stone lands in the current player's empty pit and the opposite pit on the opponent's side has some stones, collect all those stones, including the one that just landed, and place them into the current player's mancala.

- If either player's pits are entirely empty, the game concludes. The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.

## Small Board (3 Pits of 2 Stones each)

1. **play**: The `play` function allows players to take turns and make moves. The function correctly distributes stones according to the specified game rules.

2. **valid_move**: The `valid_move` function ensures that a player's chosen move is valid.

3. **winning_eval**: The `winning_eval` function determines when the game is over and which player wins. The game ends when any player's pits are all empty. The winner is the player with the most stones in their mancala. If both mancalas have the same number of stones, it's a tie.

The pits are 1-indexed when displaying and picking to make a move.

## Random Player (6 Pits of 4 Stones each)

1. **Random Move Generator**: Define the `random_move_generator` that selects a random pit from the available non-empty pits for the random player. The random player should choose a move based on these criteria. \
Set the 'seed' value to ensure that the generated values remain consistent and reproducible when grading.

You may refer to these links: [How to generate random integers in Python](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/#:~:text=Random%20integer%20values%20can%20be,for%20the%20generated%20integer%20values.), [How to use seed in Python random](https://www.w3schools.com/python/ref_random_seed.asp)


The objective is to play up to **10** moves in total (5 moves by user, 5 moves by random player)

The output submitted should reflect the state of the board and the moves played.

In [17]:
from games4e import minmax_decision, alpha_beta_cutoff_search

In [18]:
import random
import time
from copy import deepcopy
import numpy as np
# random.seed(109)

In [19]:
class Mancala:
    def __init__(self, pits_per_player=3, stones_per_pit = 2):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
    
    def get_state(self):
        return self.board
    
    def get_player(self):
        return self.current_player
    
    def get_actions(self):
        choices = []
        if self.current_player == 1:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                if self.board[i] > 0:
                    choices.append(i - self.p1_pits_index[0] + 1)
        else:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                if self.board[i] > 0:
                    choices.append(i - self.p2_pits_index[0] + 1)
        return choices
        
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if not isinstance(pit, int):
            return False
        elif pit < 1 or pit > self.pits_per_player:
            return False
        
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + (pit - 1)
        else:
            pit_index = self.p2_pits_index[0] + (pit - 1) 
            
        return self.board[pit_index] > 0 
    
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """  
        choices = []
        if self.current_player == 1:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                if self.board[i] > 0:
                    choices.append(i - self.p1_pits_index[0] + 1)
        else:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                if self.board[i] > 0:
                    choices.append(i - self.p2_pits_index[0] + 1)
        if choices:
            x = random.choice(choices)
            # print(f'RANDOM CHOICE: {x}   Player: {self.current_player}')
            return x
        
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        if not self.valid_move(pit):
            #print(f"Player {self.current_player} chose pit: {pit}")
            #print("INVALID MOVE")
            return self.board
        
        if self.winning_eval() is not None:
            #print("GAME OVER")
            return self.board
        
        self.moves.append((self.current_player, pit))
        #print(f"Player {self.current_player} chose pit: {pit}")
        
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + (pit - 1)
            mancala = self.p1_mancala_index
            opp_mancala = self.p2_mancala_index
            pits = list(range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
            opp_pits = list(range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
        else:
            pit_index = self.p2_pits_index[0] + (pit - 1)
            mancala = self.p2_mancala_index
            opp_mancala = self.p1_mancala_index
            pits = list(range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
            opp_pits = list(range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
            
        stones = self.board[pit_index]
        self.board[pit_index] = 0
        curr = pit_index
        
        while stones > 0:
            curr = (curr + 1) % len(self.board)
            
            if curr == opp_mancala:
                continue
            else:
                self.board[curr] += 1
                stones -= 1
            
        if curr == mancala or curr in opp_pits or (curr in pits and self.board[curr] > 1):
            pass
        elif curr in pits and self.board[curr] == 1:
            opp = opp_pits[-(curr - pits[0] + 1)]
            if self.board[opp] > 0:
                self.board[mancala] += self.board[opp] + self.board[curr]
                self.board[opp] = 0
                self.board[curr] = 0
                
        if self.current_player == 1:
            self.current_player = 2
        elif self.current_player == 2:
            self.current_player = 1
        
        self.winning_eval()
        return self.board
    
    # def clone(self):   # DELETE ???????????????
    #     """
    #     Creates a deep copy of the current game state.
    #     """
    #     new_game = Mancala(self.pits_per_player, 0)  # Initialize with 0 stones
    #     new_game.board = self.board.copy()
    #     new_game.current_player = self.current_player
    #     new_game.moves = self.moves.copy()
    #     new_game.p1_pits_index = self.p1_pits_index.copy()
    #     new_game.p1_mancala_index = self.p1_mancala_index
    #     new_game.p2_pits_index = self.p2_pits_index.copy()
    #     new_game.p2_mancala_index = self.p2_mancala_index
    #     return new_game
    
    # def future_state(self, pit):   # DELETE ???????????????
    #     """
    #     Simulates a move in the Mancala game without modifying the actual state.
    #     :param pit: The pit chosen by the current player (1-based index).
    #     :return: A new simulated state of the game after the move.
    #     """
    #     simulated_game = self.clone()
    #     simulated_game.play(pit)
    #     return simulated_game

    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_pit_stones = sum(self.board[i] for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)) # Added this back to make the games more determinate - less ties
        p2_pit_stones = sum(self.board[i] for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        p1 = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        p2 = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1))
        
        if p1 or p2:
            p1_score = self.board[self.p1_mancala_index] + p1_pit_stones
            p2_score = self.board[self.p2_mancala_index] + p2_pit_stones
            if p1_score > p2_score:
                #print("Player 1 wins!")
                return 1
            elif p2_score > p1_score:
                #print("Player 2 wins!")
                return 2
            else:
                #print("It's a tie!")
                return 0
                      
        return None

    def utility(self):
        """
        Utility function. It returns the difference in the mancala pits based on the current player
        """
        p1_pit_stones = sum(self.board[i] for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        p2_pit_stones = sum(self.board[i] for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        if self.current_player == 1:
            return self.board[self.p1_mancala_index] + p1_pit_stones - self.board[self.p2_mancala_index] - p2_pit_stones
        else: 
            return self.board[self.p2_mancala_index] + p2_pit_stones - self.board[self.p1_mancala_index] - p1_pit_stones
    


In [20]:
# DELTE THIS????????????????????
# 
# 
# Mancala automated game 

# game = Mancala()
# game.display_board()

# # Player 1 selects pit 1 (1-based index)
# game.play(1)
# game.display_board()

# # Player 2 selects pit 2
# game.play(2)
# game.display_board()

# # Player 1 selects pit 3
# game.play(3)
# game.display_board()

# # Player 2 selects pit 2
# game.play(2)
# game.display_board()

# # Player 1 selects pit 1
# game.play(1)
# game.display_board()

# # Printing the list of moves
# print("\nList of valid moves:")
# for move in game.moves:
#     player, pit = move
#     print(f"Player {player} selected pit {pit}")


##### PROJECT PART 3
Play 100 games of random player against random player \
What percentage of games does each player (1st or 2nd) win? \
On average, how many moves does it take to win?

In [21]:
w = []
mca = []
n = 100

for x in range(n):
    game = Mancala()
    winner = game.winning_eval()
    move_count = 0
    while winner is None:
        move = game.random_move_generator()
        move_count += 1
        if move is not None:
            game.play(move)
            winner = game.winning_eval()
        else:
            break 

    w.append(winner)
    mca.append(move_count)

p1wp = (w.count(1) / n) * 100
print(f'Player 1 Win Percentage: {p1wp:.2f}%')

p2wp = (w.count(2) / n) * 100
print(f'Player 2 Win Percentage: {p2wp:.2f}%')

p3wp = (w.count(0) / n) * 100
print(f'Tie Percentage:          {p3wp:.2f}%')

print(f'Average moves per game:  {np.mean(mca):.2f}')

Player 1 Win Percentage: 43.00%
Player 2 Win Percentage: 43.00%
Tie Percentage:          14.00%
Average moves per game:  8.63


PROJECT PART 4. Build an AI player that uses minimax to choose the best move with a variable
number of plies and a utility function we describe. What percentage of games does each player (AI or random) win?
On average, how many moves does it take to win?

In [ ]:
class MinimaxAI:
    def __init__(self, game, depth=5):
        self.game = game
        self.depth = depth
    
    def eval(self):
        return self.game.utility()
    
    def result(self, state, pit):
        new_game = Mancala()  # Create a new game instance
        new_game.board = state.copy()  # Clone the current state
        new_game.current_player = self.game.current_player
        new_game.play(pit)  # Simulate the move
        return new_game.get_state()

    def choose_move(self):

        player = game.current_player

        def max_value(state):
            if game.terminal_test(state):
                return game.utility(state, player)
            v = -np.inf
            for a in game.actions(state):
                v = max(v, min_value(game.result(state, a)))
            return v

        def min_value(state):
            if game.terminal_test(state):
                return game.utility(state, player)
            v = np.inf
            for a in game.actions(state):
                v = min(v, max_value(game.result(state, a)))
            return v

        return max(game.get_actions(), key=lambda a: min_value(game.result(game.get_state(), a))) # here is recursion. Mancala doesnt have a result but this would be the future state
        

In [23]:
# PROJECT PART 6. Build an AI player that uses Alpha-Beta to choose the best move

class AlphaBetaAI:
    def __init__(self, game, depth=5):
        self.game = game
        self.depth = depth

    def eval(self):
        return self.game.utility()

    def choose_move(self):
        def max_value(state, alpha, beta, depth):
            if state.winning_eval() is not None or depth == 0:
                return state.utility()
            v = -np.inf
            for a in state.get_actions():
                future = state.future_state(a)
                if future is not None:
                    v = max(v, min_value(future, alpha, beta, depth - 1))
                    if v >= beta:
                        return v
                    alpha = max(alpha, v)
            return v

        def min_value(state, alpha, beta, depth):
            if state.winning_eval() is not None or depth == 0:
                return state.utility()
            v = np.inf
            for a in state.get_actions():
                future = state.future_state(a)
                if future is not None:
                    v = min(v, max_value(future, alpha, beta, depth - 1))
                    if v <= alpha:
                        return v
                    beta = min(beta, v)
            return v

        best_score = -np.inf
        beta = np.inf
        best_move = None
        state = self.game
        for move in state.get_actions():
            future = state.future_state(move)
            if future is not None:
                score = min_value(future, best_score, beta, self.depth - 1)
                if score > best_score:
                    best_score = score
                    best_move = move
        return best_move


RANDOM vs MIN MAX AI \
PROJECT PART 5. Play 100 games with the random player against the minimax AI player at a depth of 5 plies. \
What percentage of games does each player (AI or random) win? \
On average, how many moves does it take to win? \
Is your AI player better than random chance? Write a paragraph or two describing or why not

In [24]:
w = []
mca = []
tavg = []
num_games = 100

for x in range(num_games):
    game = Mancala()
    minBot = MinimaxAI(game)
    num_moves = 0
    winner = None
    start = time.time()

    while not winner:
        if num_moves % 2 == 0:  # Random player is FIRST
            rand_move = game.random_move_generator()
            if game.valid_move(rand_move):
                game.play(rand_move)
                num_moves += 1
                winner = game.winning_eval()  # Check to see if there is a winner
            else:
                print("Invalid move")
        else:
            move = minBot.choose_move()
            if move is not None and game.valid_move(move):
                game.play(move)
                num_moves += 1
                winner = game.winning_eval()  # Check to see if there is a winner
            else:
                print('MinBot chose invalid move')
                assert False, "MinBot made an invalid move!"

    if winner is not None:
        w.append(winner)
        mca.append(num_moves)

        end = time.time()
        tavg.append(end - start)

p1wp = (w.count(1) / num_games) * 100
print(f'Player 1 Win Percentage: {p1wp:.2f}%')

p2wp = (w.count(2) / num_games) * 100
print(f'Player 2 Win Percentage: {p2wp:.2f}%')

p3wp = (w.count(0) / num_games) * 100
print(f'Tie Percentage:          {p3wp:.2f}%')

print(f'Average moves per game:  {np.mean(mca):.2f}')
print(f'Average time per game:   {np.mean(tavg):.2f} seconds')

AttributeError: 'Mancala' object has no attribute 'result'

In [ ]:
# PROJECT PART 7. Play 100 games with the random player against the Alpha-Beta AI player at a
# depth of 5 plies
# How long does it take for a single game to run to completion?
# What percentage of games does each player (AI or random) win?
# On average, how many moves does it take to win?
# Are your results for this part different from those for your minimax AI player?
# Write a paragraph or two describing why or why not

# TO DO:
# percentages
# avg moves over 100 games
# paragraph
# Time
assert(1==0)

game = Mancala()
ab = AlphaBetaAI(game)
num_moves = 0

while num_moves < 10:
    game.display_board()

    if num_moves % 2 == 0:
        while True:
            user_move = int(input("Enter move: "))
            if game.valid_move(user_move):
                game.play(user_move)
                num_moves += 1
                break
            else:
                print("Invalid move")
    else:
        move = ab.choose_move()
        if move is not None:
            print(f"P2 chooses pit {move}")
            game.play(move)
            num_moves += 1

    winner = game.winning_eval()
    if winner:
        print("Game has reached a winning state.")
        if winner == 0:
            print("Draw.")
        else:
            print(f"Player {winner} wins!")
        break

game.display_board()
print("Total moves played:", num_moves)


4. Build an AI player that uses minimax to choose the best move with a variable
number of plies and a utility function we describe. What percentage of games does each player (AI or random) win?
On average, how many moves does it take to win?

5. Play 100 games with the random player against the minimax AI player at a
depth of 5 plies. What percentage of games does each player (AI or random) win?
On average, how many moves does it take to win?
Is your AI player better than random chance? Write a paragraph or two describing or why not

6. Build an AI player that uses Alpha-Beta to choose the best move

7. Play 100 games with the random player against the Alpha-Beta AI player at a
depth of 5 plies
How long does it take for a single game to run to completion?
What percentage of games does each player (AI or random) win?
On average, how many moves does it take to win?
Are your results for this part different from those for your minimax AI player?
Write a paragraph or two describing why or why not

8. (Extra Credit, 10 points). Play 100 games with the random player against the
Alpha-Beta AI player at a depth of 10 plies
How long does it take for a single game to run to completion?
What percentage of games does each player (AI or random) win?
On average, how many moves does it take to win?
Does increasing the number of plies improve the play for our AI player? Why
or why not?